In [1]:
#2053635任柯睿
import numpy as np
import pandas as pd

### **导入数据**
#### 训练集为ua.base，读取数据为u_base
#### 测试集为ua.test，读取数据为u_test
#### 之后基于u_base构建用户-物品交互矩阵

In [2]:
u_base=np.loadtxt("../input/movielens100k/ml-100k/ua.base",delimiter='\t',encoding='utf-8')
u_base=u_base[:,:-1]
u_test=np.loadtxt("../input/movielens100k/ml-100k/ua.test",delimiter='\t',encoding='utf-8')
u_test=u_test[:,:-1]
user_num=943
item_num=1682
user_item_mx=np.zeros((user_num,item_num),dtype=np.int32)
user_user_mx=np.zeros((user_num,user_num),dtype=np.int32)
#构建用户-物品交互矩阵
for i in range(len(u_base)):
    user_item_mx[int(u_base[i][0])-1][int(u_base[i][1])-1]=int(u_base[i][2])
u_base

array([[1.000e+00, 1.000e+00, 5.000e+00],
       [1.000e+00, 2.000e+00, 3.000e+00],
       [1.000e+00, 3.000e+00, 4.000e+00],
       ...,
       [9.430e+02, 1.188e+03, 3.000e+00],
       [9.430e+02, 1.228e+03, 3.000e+00],
       [9.430e+02, 1.330e+03, 3.000e+00]])

### **基于点击的User-CF预测**
#### 使用Jaccard相似系数得到不同用户之间的关联度，并据此计算u_test中用户点击对应物品的概率
#### 由于使用全部数据运行时间较长，我选择100个用户和100个物品，并对u_test前100条进行预测，

In [3]:
#计算相关度
def JaccardCul(u1,u2,item_num):
    up=down=0
    for i in range(item_num):
        if user_item_mx[u1][i]>0 and user_item_mx[u2][i]>0:
            up+=1
        if user_item_mx[u1][i]>0 or user_item_mx[u2][i]>0:
            down+=1
    if down!=0:
        return up/down
    else:
        return 0
    

In [4]:
#基于用户点击的User-CF
def User_CF_Click(user_num,item_num,test_num):
    item_scores=[]
    for i in range(test_num):
        r=num=0.0
        user_id=int(u_test[i][0])
        item_id=int(u_test[i][1])
        for j in range(user_num):
            if j!=user_id-1 and user_item_mx[j][item_id-1]>0:
                num+=1
                r+=JaccardCul(user_id-1,j,item_num)
        if num!=0:
            item_scores.append(r/num)
        else:
            item_scores.append(0)
    return item_scores

In [5]:
print("开始进行基于点击的User-CF预测")
item_list_click=User_CF_Click(100,100,100)
item_list_click[0:10]

开始进行基于点击的User-CF预测


[0.2401051329622758,
 0.431377264710598,
 0.45750017178588614,
 0.2387775556958179,
 0.38854789310004195,
 0.4355066702005478,
 0.3559825054670415,
 0.29352125389078126,
 0.30237274110109424,
 0.2854554287928176]

### **基于评分的User-CF预测**
#### 使用PersonCorrelavation相似系数得到不同用户之间的关联度，并据此预测出u_test中用户对物品的评分
#### 我选择全部用户和物品，并对u_test前100条进行预测，结果使用MAE函数和RMSE函数进行评估。

In [6]:
ratings_mean=[]
#建立字典存储两个用户的交叉物品集合
user_item_id=[]
for u in range(user_num):
    uitem=[]
    total=0
    num=0
    for i in range(item_num):
        if user_item_mx[u][i]>0:
            num+=1
            total+=user_item_mx[u][i]
            uitem.append(i)
    if num!=0:
        ratings_mean.append(total/num)
    else:
        ratings_mean.append(0)
    user_item_id.append(uitem)

In [7]:
#计算相关度
def PersonCorrelavationCul(u1,u2):
    s1=s2=s3=0.0
    u1item=user_item_id[u1]
    u2item=user_item_id[u2]
    item_list= list(set(u1item) & set(u2item))
    for i in item_list:
        s1+=(user_item_mx[u1][i]-ratings_mean[u1])*(user_item_mx[u2][i]-ratings_mean[u2])
        s2+=(user_item_mx[u1][i]-ratings_mean[u1])*(user_item_mx[u1][i]-ratings_mean[u1])
        s3+=(user_item_mx[u2][i]-ratings_mean[u2])*(user_item_mx[u2][i]-ratings_mean[u2])
    if s2*s3!=0: 
        return s1/(np.sqrt(s2)*np.sqrt(s3))
    else:
        return 0

In [8]:
u1item=user_item_id[0]
u2item=user_item_id[3]
item_list= list(set(u1item) & set(u2item))
item_list

[257, 10, 209, 270]

In [9]:
#基于用户对物品评分的User-CF
def User_CF_Ratings(user_num,test_num=len(u_test)):
    item_scores=[]
    for i in range(test_num):
        dr_up=dr_down=0.0
        user_id=int(u_test[i][0])
        item_id=int(u_test[i][1])
        for j in range(user_num):
            if j!=user_id-1 and user_item_mx[j][item_id-1]>0:
                if user_user_mx[user_id][j]!=0:
                    s=user_user_mx[user_id-1][j]
                else:
                    s=PersonCorrelavationCul(user_id-1,j)
                    user_user_mx[user_id-1][j]=user_user_mx[j][user_id-1]=s
                dr_up+=s*(user_item_mx[j][item_id-1]-ratings_mean[j])
                dr_down+=s
        r=ratings_mean[user_id-1]+(dr_up/dr_down)
        if r>5:r=5
        if r<0:r=0
        item_scores.append(r)
    return item_scores

In [10]:
print("开始进行基于评分的User-CF预测")
item_list_ratings=User_CF_Ratings(user_num,100)
item_list_ratings[0:10]

开始进行基于评分的User-CF预测


[3.5314716846633862,
 3.523963175642749,
 4.001296999385892,
 3.70662074773827,
 2.8357314885533804,
 3.555782717003127,
 4.223049977509987,
 4.270654418177749,
 3.850747346607824,
 3.9509954414645576]

In [11]:
for i in range(len(item_list_ratings)):
    item_list_ratings[i]=round(item_list_ratings[i])

### **定义评估函数**
#### 定义MAE函数和RMSE函数，用于评估预测结果的准确性

In [12]:
#计算MAE
def MAE(item_list):
    tot=0.0
    n=len(item_list)
    for i in range(n):
        r=u_test[i][2]
        p=item_list[i]
        if r!=0: tot+=np.fabs(r-p)
    return tot/n

In [13]:
#计算RMSE
def RMSE(item_list):
    tot=0.0
    n=len(item_list)
    for i in range(n):
        r=u_test[i][2]
        p=item_list[i]
        if r!=0:tot+=(r-p)*(r-p)
    return np.sqrt(tot/n)

In [14]:
#输出结果
mae=MAE(item_list_ratings)
rmse=RMSE(item_list_ratings)
print('MAE={0:.4f},RMSE={1:.4f}'.format(mae,rmse))

MAE=0.8000,RMSE=1.2884


In [15]:
#用于具体对比预测结果和测试结果
n=len(item_list_ratings)
real_list=[]
precision_list=[]
for i in range(n):
    r=u_test[i][2]
    if r!=0:
        real_list.append(r)
        precision_list.append(item_list_ratings[i])
print(real_list)
print(precision_list)

[4.0, 4.0, 4.0, 3.0, 2.0, 4.0, 5.0, 3.0, 5.0, 4.0, 4.0, 5.0, 5.0, 3.0, 3.0, 3.0, 4.0, 4.0, 3.0, 1.0, 1.0, 2.0, 2.0, 5.0, 4.0, 1.0, 3.0, 1.0, 1.0, 3.0, 5.0, 4.0, 3.0, 4.0, 5.0, 5.0, 5.0, 3.0, 4.0, 5.0, 4.0, 3.0, 4.0, 3.0, 1.0, 2.0, 3.0, 1.0, 1.0, 1.0, 5.0, 4.0, 3.0, 3.0, 5.0, 2.0, 2.0, 4.0, 5.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 3.0, 4.0, 1.0, 1.0, 3.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 4.0, 3.0, 5.0, 4.0, 4.0, 5.0, 3.0, 5.0, 3.0, 4.0, 5.0, 5.0, 5.0]
[4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 4, 4, 3, 2, 0, 2, 1, 3, 5, 5, 5, 5, 2, 2, 5, 3, 4, 1, 5, 0, 5, 5, 5, 5, 3, 3, 3, 4, 2, 2, 2, 1, 1, 2, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 5, 4, 5, 4, 3, 3, 4, 2, 4, 4, 4, 4, 5, 5, 5, 4, 4, 5, 0, 4, 4, 5, 4, 5, 4, 4, 5, 5, 5]
